## Importation of most impotant libraries

In [1]:
#Importing library
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.utils import *
from keras.initializers import *
import tensorflow as tf
import time, random

## Definition of Hyperparameters

In [2]:
#Hyperparameters
batch_size = 64
latent_dim = 256
num_samples = 10000

## Vectorization of input data to feature vectors

In the vectorization process,the collection of text documents is converted into feature vectors

In [3]:
#Vectorize the data.
input_texts = []
target_texts = []
input_chars = set()
target_chars = set()

with open('eng_yala.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_chars:
            input_chars.add(char)
    for char in target_text:
        if char not in target_chars:
            target_chars.add(char)

input_chars = sorted(list(input_chars))
target_chars = sorted(list(target_chars))
num_encoder_tokens = len(input_chars)
num_decoder_tokens = len(target_chars)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#Print size
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 259
Number of unique input tokens: 40
Number of unique output tokens: 67
Max sequence length for inputs: 36
Max sequence length for outputs: 47


## defining Encoder and Decoder

 Define the input data encoder and decoder and the target data for the decoder.

In [4]:
#Define data for encoder and decoder
input_token_id = dict([(char, i) for i, char in enumerate(input_chars)])
target_token_id = dict([(char, i) for i, char in enumerate(target_chars)])

encoder_in_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_in_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_in_data[i, t, input_token_id[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_in_data[i, t, target_token_id[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_id[char]] = 1.

Below lines of codes will define the input sequence for the encoder defined above and process this sequence. After that, an initial state will be set up for the decoder using ‘encoder_states’

In [5]:
#Define and process the input sequence
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
#We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

#Using `encoder_states` set up the decoder as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

The below line of code will define the final model that will turn `encoder_in_data` & `decoder_in_data` into `decoder_target_data`.

In [6]:
#Final model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

After defining the final model, we will check it by its summary, data shape and a visualization.

In [7]:
#Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 40)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 67)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 304128      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  331776      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [8]:
#Model data Shape
print("encoder_in_data shape:",encoder_in_data.shape)
print("decoder_in_data shape:",decoder_in_data.shape)
print("decoder_target_data shape:",decoder_target_data.shape)

encoder_in_data shape: (259, 36, 40)
decoder_in_data shape: (259, 47, 67)
decoder_target_data shape: (259, 47, 67)


In [9]:
#Visuaize the model
tf.keras.utils.plot_model(model,show_shapes=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


## compile and train the model

In [10]:
#Compiling and training the model
epochs =100
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy')

model.fit([encoder_in_data, decoder_in_data], decoder_target_data, batch_size = batch_size, epochs=epochs, validation_split=0.2)

Epoch 1/100
4/4 [==============================] - 5s 626ms/step - loss: 1.0964 - val_loss: 0.9530
Epoch 2/100
4/4 [==============================] - 1s 177ms/step - loss: 0.9954 - val_loss: 0.9486
Epoch 3/100
4/4 [==============================] - 1s 168ms/step - loss: 0.9901 - val_loss: 0.9132
Epoch 4/100
4/4 [==============================] - 1s 164ms/step - loss: 0.9243 - val_loss: 0.9094
Epoch 5/100
4/4 [==============================] - 1s 155ms/step - loss: 0.9346 - val_loss: 0.8937
Epoch 6/100
4/4 [==============================] - 1s 161ms/step - loss: 0.8801 - val_loss: 0.8961
Epoch 7/100
4/4 [==============================] - 1s 159ms/step - loss: 0.9277 - val_loss: 0.8965
Epoch 8/100
4/4 [==============================] - 1s 157ms/step - loss: 0.8972 - val_loss: 0.8893
Epoch 9/100
4/4 [==============================] - 1s 160ms/step - loss: 0.9153 - val_loss: 0.8847
Epoch 10/100
4/4 [==============================] - 1s 162ms/step - loss: 0.9110 - val_loss: 0.8865
Epoch 11/

4/4 [==============================] - 1s 150ms/step - loss: 0.7509 - val_loss: 0.8479
Epoch 84/100
4/4 [==============================] - 1s 153ms/step - loss: 0.7438 - val_loss: 0.8512
Epoch 85/100
4/4 [==============================] - 1s 149ms/step - loss: 0.7552 - val_loss: 0.8510
Epoch 86/100
4/4 [==============================] - 1s 151ms/step - loss: 0.7587 - val_loss: 0.8544
Epoch 87/100
4/4 [==============================] - 1s 153ms/step - loss: 0.7319 - val_loss: 0.8514
Epoch 88/100
4/4 [==============================] - 1s 161ms/step - loss: 0.7412 - val_loss: 0.8483
Epoch 89/100
4/4 [==============================] - 1s 175ms/step - loss: 0.7507 - val_loss: 0.8483
Epoch 90/100
4/4 [==============================] - 1s 163ms/step - loss: 0.7551 - val_loss: 0.8475
Epoch 91/100
4/4 [==============================] - 1s 187ms/step - loss: 0.7411 - val_loss: 0.8536
Epoch 92/100
4/4 [==============================] - 1s 192ms/step - loss: 0.7566 - val_loss: 0.8480
Epoch 93/100


define the sample model using the parameters of the trained model to test language translation.

In [11]:
#Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [16]:
model.save('lstm_model.h5')
encoder_model.save('encoder_model.h5')
decoder_model.save('decoder_model.h5')

The below part of codes will define the decode sequence for the text that we will pass to the model as the input sequence. This
could be understood as the module for translating the input language into the target language. In this part, the input sequence
is encoded into the state vectors. The state vector and the target sequence is passed to the decoder and it produces the 
prediction for the next character. Using these predictions, the next character is sampled and it is appended to the target
sequence. This process is repeated to generate until the end of the sequence.

In [13]:
reverse_input_char_index = dict((i, char) for char, i in input_token_id.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_id.items())

#Define Decode Sequence
def decode_sequence(input_seq):
    #Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    #Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Get the first character of target sequence with the start character.
    target_seq[0, 0, target_token_id['\t']] = 1.

    #Sampling loop for a batch of sequences
    #(to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        #Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        #Exit condition: either hit max length
        #or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        #Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        #Update states
        states_value = [h, c]

    return decoded_sentence

Finally, we will check our model to decode the input sequence into the target sequence, i.e., translate the English sentences
into the French sentences.

In [14]:
for seq_index in range(15):
    input_seq = encoder_in_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: a        lot!
Decoded sentence: obi̍                                            
-
Input sentence: A week later!
Decoded sentence: obi̍                                            
-
Input sentence: also!        
Decoded sentence: obi̍                                            
-
Input sentence: always       
Decoded sentence: obi̍                                            
-
Input sentence: ancestors   
Decoded sentence: obi̍                                            
-
Input sentence: and         
Decoded sentence: obi̍                                            
-
Input sentence: and you     
Decoded sentence: obi̍                                            
-
Input sentence: angels      
Decoded sentence: obi̍                                            
-
Input sentence: another     
Decoded sentence: obi̍                                            
-
Input sentence: another place
Decoded sentence: obi̍                                            
-
Input sentenc

## References: 

1.Ilya Sutskever et al, ‘Sequence to Sequence Learning with Neural Networks’, arXiv.org.

2.K Cho et al, ‘Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation’, arXiv.org.

3.Keras tutorial on ‘Sequence to sequence example in Keras (character-level)’.

4.Keras tutorial on ‘A ten-minute introduction to sequence-to-sequence learning in Keras’

Thanks